# Project 3
## Team members Arick Smith And Treyson Grange

## Ideas

- Scrape rolling stones for most mentioned artists
- Scrape cooking recipes, find trends in ingredients and how expensive the recipes are
- Working with the data creation from UCI mountain Bike world racing data set. (CHOOSEN)


## TASKS
- need to use the requests, Beautiful Soup, html5lib, and/or API libraries (e.g. Twitter, RottenTomatoes) to get web content.
- Your analysis should include at least four techniques that we've discussed in class, and they should all contribute in a meaningful way to telling the story. 
- Make Project Report
- Make Presentation



In [6]:
# Common imports for the parsing of website data
from bs4 import BeautifulSoup
import requests
import re


#Common imports for use in the data visualization
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import csv





Working on the parsing of website data

In [7]:
call_web = "https://ucimtbworldseries.com/teams"

site = requests.get(call_web)

og_web_html = ''

og_web_html = site.text
print(og_web_html[:500])

<!doctype html>
<!-- uci mtb world series -->
<html lang="">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <meta name="csrf-token" content="a0cv3TA0RmihqHFITWrVx3yJWIshF0MAWO0peQKJ">

    <title>UCI Mountain Bike World Series | Teams</title><meta name="description" content="For all the latest news, action and events on the UCI Mountain Bike World Series calendar. Welcome to our UCI Mountain Bike World Series HQ"><meta name="keyword


In [8]:
team_matches = len(list(re.finditer('/teams/', str(og_web_html))))

print("the total count of teams is likely half this number ", team_matches)



the total count of teams is likely half this number  392


In [9]:
soup = BeautifulSoup(og_web_html)


#Using the same code as the inclass example I found that the items classed as flex-shrink-0 would have our hyperlinks
teams = [a['href'] for div in soup.find_all('div', {'class':'flex-shrink-0'}) for a in div.find_all('a')]
# Removing the home dir
teams.remove('/')
print(teams)
print(len(teams))

#There are a total of 196 teams involved


['https://ucimtbworldseries.com/teams/555-gravity-racing', 'https://ucimtbworldseries.com/teams/abetone-fullout-gravity', 'https://ucimtbworldseries.com/teams/alpecin-deceuninck', 'https://ucimtbworldseries.com/teams/backyard-racing-factory-team', 'https://ucimtbworldseries.com/teams/bad-school-team-cd', 'https://ucimtbworldseries.com/teams/bear-national-team', 'https://ucimtbworldseries.com/teams/berria-vittoria-factory-team', 'https://ucimtbworldseries.com/teams/beyond-racing', 'https://ucimtbworldseries.com/teams/bh-coloma-team', 'https://ucimtbworldseries.com/teams/bh-enduro-racing-team', 'https://ucimtbworldseries.com/teams/bh--wallonie-mtb-team', 'https://ucimtbworldseries.com/teams/bike-team-solothurn', 'https://ucimtbworldseries.com/teams/bixs-performance-race-team', 'https://ucimtbworldseries.com/teams/buff-megamo-team', 'https://ucimtbworldseries.com/teams/caloi-henrique-avancini-racing', 'https://ucimtbworldseries.com/teams/cannondale-enduro', 'https://ucimtbworldseries.com/

In [10]:
#Going to enter each team and make a compilition of the members that compose it (Getting each url to a team member)
players_links = []

for team in teams[:25]:
    cur_team = requests.get(team)
    if cur_team.status_code == 200:
        sur_team_text = cur_team.text

        team_soup = BeautifulSoup(sur_team_text)

        current_links = [a['href'] for div in team_soup.find_all('div', {'id':'team-riders'}) for a in div.find_all('a')]

        #Breaking apart links to have in one massive player link list
        for i in current_links:
            players_links.append(i)

#All player links
players_links


['https://ucimtbworldseries.com/athletes/56172',
 'https://ucimtbworldseries.com/athletes/56173',
 'https://ucimtbworldseries.com/athletes/56174',
 'https://ucimtbworldseries.com/athletes/25437',
 'https://ucimtbworldseries.com/athletes/33592',
 'https://ucimtbworldseries.com/athletes/22813',
 'https://ucimtbworldseries.com/athletes/21925',
 'https://ucimtbworldseries.com/athletes/56175',
 'https://ucimtbworldseries.com/athletes/56176',
 'https://ucimtbworldseries.com/athletes/56177',
 'https://ucimtbworldseries.com/athletes/56178',
 'https://ucimtbworldseries.com/athletes/56179',
 'https://ucimtbworldseries.com/athletes/56180',
 'https://ucimtbworldseries.com/athletes/56181',
 'https://ucimtbworldseries.com/athletes/56182',
 'https://ucimtbworldseries.com/athletes/56183',
 'https://ucimtbworldseries.com/athletes/56184',
 'https://ucimtbworldseries.com/athletes/56185',
 'https://ucimtbworldseries.com/athletes/56186',
 'https://ucimtbworldseries.com/athletes/56188',
 'https://ucimtbworl

In [11]:
# getTopElements
# input HTML Soup
# Output: Team, Nationality, age, format, category
def getTopElements(rider_soup):
    divList = rider_soup.find('div', class_='flow-root bg-white')# this i gets the list at the top of the page
    ul_element = divList.find('ul', class_='divide-y divide-gray-200 bg-white m-4')
    output = []
    output.append(rider_soup.find('p', class_='truncate text-xl font-headline text-gray-900 uppercase').text)
    for li_element in ul_element.find_all('li'):
        for div_element in li_element.find_all('div'):
            if div_element.find('svg'):
                continue
            if not div_element.find('div'):
                continue
            
            for p_element in div_element.find_all('p'):
                if p_element.find('span'):
                    continue
                output.append(p_element.text)

            

    return output


def getRiderResults(rider_soup):
    allPlacements = []
    table = rider_soup.find('table')
    tbody = table.find('tbody')
    for tr in tbody.find_all('tr'):
        th_element = tr.find('td', class_='whitespace-nowrap py-4 pl-4 pr-3 text-center text-sm font-medium text-gray-900 sm:pl-0')
        th_text = th_element.text.strip().rstrip('\n')
        allPlacements.append(th_text)
    
    return allPlacements
        
        
        


In [ ]:
#The following will create a rider entry to a csv file
#This will contain a few of the following peices of data: NAME,AGE,NATIONALITY,CATEGORY, etc...
with open('riders.csv', 'w', newline='', encoding='utf-8') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',')
    filewriter.writerow(['Name', 'Age', 'Nationality','Team', 'Category', 'Formats', 'AVG_Place', 'TTL_First', 'TTL_Top_Ten', 'TTL_Races_2023', 'AVG_T_Behind_First', 'XCOWCP', 'XCCWCP'])
    
    for rider in players_links:
        # print(rider)
        cur_rider = requests.get(rider)
        if cur_rider.status_code == 200:
            cur_rider_html = cur_rider.text
            rider_soup = BeautifulSoup(cur_rider_html)

            # For name, age, nationality, category, and formats
            output = getTopElements(rider_soup)
            print(output)
            # For placements of every race
            allPlacements = getRiderResults(rider_soup)
            
            # For AVG_Place
            total = 0
            while "DNF" in allPlacements:
                allPlacements.remove("DNF")  # Remove "DNF" from the list
            while "DSQ" in allPlacements:
                allPlacements.remove("DSQ")  # Remove "DNF" from the list
            while "DNS" in allPlacements:
                allPlacements.remove("DNS")  # Remove "DNS" from the list
            for num in allPlacements:
                try:
                    total += float(num)
                except ValueError:
                    pass  # Ignore non-numeric values like "DNS"
            
            count = len(allPlacements)
            #For TTL_First and TTL_Top_Ten
            TTLFirstCounter = 0
            TTLTop10Counter = 0
            for item in allPlacements:
                if int(item)<= 1:
                    TTLFirstCounter += 1
                if int(item)<= 10:
                    TTLTop10Counter += 1


            formatCleaned = output[4].split(',')
            categoryCleaned = output[5].split(',')
            if count == 0:
                average = 0
            else:    
                average = round(total / count)


            try:
                filewriter.writerow([output[0].rstrip('\n').strip(), output[5].rstrip('\n').strip(), output[3].rstrip('\n').strip(), output[1].rstrip("\n").strip(), output[9].rstrip('\n').strip(), output[7].strip().rstrip('\n'), average, TTLFirstCounter, TTLTop10Counter, len(allPlacements), 0, 0, 0])
            except IndexError:
                pass
            
    

# Transition from gathering Data to Interpreting the data
## Questions we seek to answer

- Whos the nation to beat in world series Mountain Biking?
- Does America Beat out the British?
- What contributes to riders getting First Place?
- Does a DNF mean they are an often poor racer? (Bottom half of leaderboard)

In [23]:
#Reading in the data from our created data set. This ensures it works with the code without the need to make the CSV file every time

rider_df = pd.read_csv('riders.csv', encoding='utf-8')
rider_df.head(8)

,Name,Age,Nationality,Team,Category,Formats,AVG_Place,TTL_First,TTL_Top_Ten,TTL_Races_2023,AVG_T_Behind_First,XCOWCP,XCCWCP
0,Luke WILLIAMSON,22,United Kingdom,555 GRAVITY RACING,Men Elite,NaN,66,0,0,10,0,0,0
1,Ryan BRANNEN,21,United Kingdom,555 GRAVITY RACING,Men Elite,NaN,125,0,0,3,0,0,0
2,Douglas GOODWILL,20,United Kingdom,555 GRAVITY RACING,Men Elite,NaN,97,0,0,6,0,0,0
3,Nadine ELLECOSTA,25,Italy,ABETONE ANCILLOTTI VITTORIA FACTORY TEAM,Women Elite,EDR,14,0,1,6,0,0,0
4,Tommaso ACERBI,27,Italy,ABETONE ANCILLOTTI VITTORIA FACTORY TEAM,Men Elite,EDR,82,0,0,3,0,0,0
5,Tommaso FRANCARDO,27,Italy,ABETONE ANCILLOTTI VITTORIA FACTORY TEAM,Men Elite,EDR,37,0,2,8,0,0,0
6,Nicola GROTTI,24,Italy,ABETONE ANCILLOTTI VITTORIA FACTORY TEAM,Men Elite,EDR,92,0,0,2,0,0,0
7,Ceylin Del Carmen ALVARADO,26,Netherlands,ALPECIN-DECEUNINCK,Women Elite,XCO\r\n ...,40,0,0,3,0,0,0


In [25]:
# Grab all of the rider info for the brits for later use

rider_UK = rider_df[rider_df['Nationality'] == "United Kingdom"]
rider_UK.head(5)


,Name,Age,Nationality,Team,Category,Formats,AVG_Place,TTL_First,TTL_Top_Ten,TTL_Races_2023,AVG_T_Behind_First,XCOWCP,XCCWCP
0,Luke WILLIAMSON,22,United Kingdom,555 GRAVITY RACING,Men Elite,NaN,66,0,0,10,0,0,0
1,Ryan BRANNEN,21,United Kingdom,555 GRAVITY RACING,Men Elite,NaN,125,0,0,3,0,0,0
2,Douglas GOODWILL,20,United Kingdom,555 GRAVITY RACING,Men Elite,NaN,97,0,0,6,0,0,0


In [26]:
# Grab all of the rider info for the yanks for later use

rider_USA = rider_df[rider_df['Nationality'] == "United States"]
rider_USA.head(5)


,Name,Age,Nationality,Team,Category,Formats,AVG_Place,TTL_First,TTL_Top_Ten,TTL_Races_2023,AVG_T_Behind_First,XCOWCP,XCCWCP
15,Lasse KONECNY,21,United States,BEAR NATIONAL TEAM,Men U23,XCO\r\n ...,43,0,0,3,0,0,0
16,Daxton MOCK,23,United States,BEAR NATIONAL TEAM,U23 Men,XCO\r\n ...,102,0,0,2,0,0,0
17,Cayden Parker,20,United States,BEAR NATIONAL TEAM,U23 Men,XCO\r\n ...,57,0,0,5,0,0,0
18,Cobe FREEBURN,23,United States,BEAR NATIONAL TEAM,U23 Men,XCO\r\n ...,0,0,0,0,0,0,0
19,Carson HAMPTON,21,United States,BEAR NATIONAL TEAM,Men U23,XCO\r\n ...,44,0,0,3,0,0,0


In [22]:
# Before only comparing the US and UK Lets find out which nation is running the best track record 

nationalities = rider_df['Nationality'].unique()
nationalities

array(['United Kingdom', 'Italy', 'Netherlands', 'New Zealand', 'Germany',
       'Belgium', 'Switzerland', 'United States', 'France',
       'Czech Republic', 'Canada'], dtype=object)

In [28]:
first_place_count = {}

for nation in nationalities:
    nation_df = rider_df[rider_df['Nationality'] == nation]
    nation_first_place = nation_df['TTL_First'].sum()
    nation_TTL_races = nation_df['TTL_Races_2023'].sum()

    percent_of_races_first = nation_first_place/nation_TTL_races * 100

    print(f'Total First places is: {nation_first_place}|| In total {nation} racers competed in {nation_TTL_races} in 2023. As a nation they get first {percent_of_races_first}% of the time')


Total First places is: 0|| In total United Kingdom racers competed in 19 in 2023. As a nation they get first 0.0% of the time
Total First places is: 0|| In total Italy racers competed in 19 in 2023. As a nation they get first 0.0% of the time
Total First places is: 4|| In total Netherlands racers competed in 19 in 2023. As a nation they get first 21.052631578947366% of the time
Total First places is: 0|| In total New Zealand racers competed in 7 in 2023. As a nation they get first 0.0% of the time
Total First places is: 0|| In total Germany racers competed in 7 in 2023. As a nation they get first 0.0% of the time
Total First places is: 0|| In total Belgium racers competed in 8 in 2023. As a nation they get first 0.0% of the time
Total First places is: 0|| In total Switzerland racers competed in 11 in 2023. As a nation they get first 0.0% of the time
Total First places is: 0|| In total United States racers competed in 41 in 2023. As a nation they get first 0.0% of the time
Total First p